In [2]:
import sys
import os
import openai
# from openai import OpenAI

sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file


openai.api_key = os.environ["OPENAI_API_KEY"]
# client = OpenAI()

In [3]:
def upload_file(local_path):
    file=client.files.create(
        file=open(local_path, 'rb'),
        purpose='assistants'
    )

def create_assistant(file):
    assistant = client.beta.assistants.create(
        name="Test assistant",
        instructions="Test instructions",
        tools=[{"type":"retrieval"}],
        model="gpt-4-1106-preview",
        file_ids=[file.id],
    )
    return assistant